# View sample predictions

Trained model from checkpoint

In [1]:
import argparse
import random
import sys
import warnings

from transformers.models.mt5 import MT5Tokenizer
import pandas as pd

sys.path.append('..')  # Allow import of project packages
from text_summarizer.data.motions_data_module import MotionsDataModule
from text_summarizer.models import t5
from text_summarizer.lit_models import MT5LitModel
from text_summarizer.util import summarize

/home/erik/miniconda3/envs/swe-parl-mot-summarization/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = MotionsDataModule(args=argparse.Namespace())
dataset.prepare_data()
dataset.setup()
print(dataset)

Filtered 618 rows with missing values.
Filtered 7444 texts shorter than 150 characters.
Number of rows remaining: 161989
Using 161989 of 161989 examples.
Train: 113392, Val: 24298, Test: 24299


In [3]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 500)
 
MODEL_PATH = "../training/logs/lightning_logs/version_17/checkpoints/epoch=009-val_loss=0.000-val_cer=0.000.ckpt"
RANDOM_STATE = 1

### Load model

In [4]:
model = t5.MT5(
    data_config=dataset.config(),
    args=argparse.Namespace()
)

lit_model = MT5LitModel.load_from_checkpoint(
    checkpoint_path=MODEL_PATH,
    model=model
)
lit_model.freeze()

tokenizer = MT5Tokenizer.from_pretrained(model.model_name)

### Load data

In [6]:
def summarize(model, text, tokenizer, text_max_num_tokens, summary_max_num_tokens):
    text_encoding = tokenizer(
        text,
        max_length=text_max_num_tokens,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )

    generated_ids = model.model.generate(
        input_ids=text_encoding["input_ids"],
        attention_mask=text_encoding["attention_mask"],
        max_length=summary_max_num_tokens,
        num_beams=2,
        repetition_penalty=5.0,
        length_penalty=2.0,
        early_stopping=True,
    )

    preds = [
        tokenizer.decode(
            gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        for gen_id in generated_ids
    ]

    return "".join(preds)

In [23]:
def show_sample_pred(tokenizer):
    sample_index = random.randint(0, len(dataset.data_test.data))
    text = dataset.data_test.data[sample_index]
    true_summary = dataset.data_test.targets[sample_index]
    model_summary = summarize(
        model=model,
        text=text,
        tokenizer=tokenizer,
        text_max_num_tokens=512,
        summary_max_num_tokens=64
    )
    print("Motion text:")
    print(50*"-")
    print(text[:500])
    print(50*"-")
    print("Actual title:")
    print(true_summary)
    print(50*"-")
    print("Predicted title:")
    print(model_summary)
    print(50*"-")

show_sample_pred(lit_model.tokenizer)

Motion text:
--------------------------------------------------
I årets budgetproposition (bil. 10) föreslås att en processindustrilinje om 60 poäng förläggs till högskolan i Sundsvall/Härnösand. Universitets- och högskoleämbetet har också i sin informationsbroschyr om utbildningslinjer inom den yrkestekniska högskolan tagit upp nämnda linje. 1 både budgetpropositionen och informationsbroschyren framställs linjen så att den också skall vara inriktad mot pappers- och massaindustrin. En sådan utbildning finns emellertid redan vid Lunds universitet förlagd till
--------------------------------------------------
Actual title:
Verksamheten vid de yrkestekniska högskolorna i Markaryd och Sundsvall/Härnösand
--------------------------------------------------
Predicted title:
med anledning av prop. 2021/22:168 Ökad tillgång till spelmarknad
--------------------------------------------------
